In [ ]:
#Drive ile bağlantı
from google.colab import drive
drive.mount('/content/gdrive') 

Mounted at /content/gdrive


In [ ]:
!nvidia-smi #GPU kontrol

Sun May 23 12:08:49 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
#Darknet indirilmesi
!git clone https://github.com/AlexeyAB/darknet
%cd darknet

!ls

Cloning into 'darknet'...
remote: Enumerating objects: 15050, done.
remote: Counting objects: 100% (48/48), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 15050 (delta 16), reused 34 (delta 14), pack-reused 15002
Receiving objects: 100% (15050/15050), 13.45 MiB | 25.26 MiB/s, done.
Resolving deltas: 100% (10214/10214), done.
/content/darknet
3rdparty		darknet_video.py       net_cam_v4.sh
build			data		       README.md
build.ps1		image_yolov3.sh        results
cfg			image_yolov4.sh        scripts
cmake			include		       src
CMakeLists.txt		json_mjpeg_streams.sh  vcpkg.json
DarknetConfig.cmake.in	LICENSE		       video_yolov3.sh
darknet_images.py	Makefile	       video_yolov4.sh
darknet.py		net_cam_v3.sh


In [ ]:
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile

In [ ]:
#Darknetin ayarları yapılarak MAKE ediliyor
!make 

In [ ]:
#Ağırlık, ayarlar, etiket isimleri kopyalanıyor
!cp /content/gdrive/MyDrive/project/obj.names /content/darknet/
!cp /content/gdrive/MyDrive/project/obj.data /content/darknet/
!cp /content/gdrive/MyDrive/project/yolov4.cfg /content/darknet/
!cp /content/gdrive/MyDrive/project/yolov4.weights /content/darknet/

In [ ]:
#Eğitim için
!mkdir /content/train # test train klasörleri oluşturuluyor
!mkdir /content/test

In [ ]:
#Eğitim için
from shutil import copyfile
import os

train_path = f"/content/gdrive/MyDrive/project/veriler/" # train dataları kopyalanıyor

train_files = os.listdir(train_path)

train_dest_path = "/content/train/"

for index, file_name in enumerate(train_files):
  if index % 10 == 0:
    print(f"Train Dosyaları kopyalanıyor: {index}/{len(train_files)}")
  copyfile(train_path + file_name, train_dest_path + file_name)  


In [ ]:
from shutil import copyfile
import os

train_path = f"/content/gdrive/MyDrive/project/test/" # train dataları kopyalanıyor

train_files = os.listdir(train_path)

train_dest_path = "/content/test/"

for index, file_name in enumerate(train_files):
  if index % 10 == 0:
    print(f"Train Dosyaları kopyalanıyor: {index}/{len(train_files)}")
  copyfile(train_path + file_name, train_dest_path + file_name)

In [ ]:
#Eğitim için
import glob, os, random, time # train.txt ve test.txt oluşturuluyor

test = "\n".join(["/content/test/" + file_name for file_name in os.listdir("/content/test/") if file_name[-3:] == "png"])
with open('/content/darknet/test.txt', 'w') as f:   
  f.write(test)

train = "\n".join(["/content/train/" + file_name for file_name in os.listdir("/content/train/") if file_name[-3:] == "png"])
with open('/content/darknet/train.txt', 'w') as t_f:   
  t_f.write(train)

In [ ]:
# Eğitim için
!./darknet detector train "/content/darknet/obj.data" "/content/darknet/yolov4.cfg" "/content/darknet/yolov4.weights" -dont_show -map

Streaming output truncated to the last 5000 lines.
 5864: 6.713333, 11.364264 avg loss, 0.000026 rate, 0.730319 seconds, 46912 images, 0.058905 hours left
Loaded: 0.000045 seconds
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 139 Avg (IOU: 0.000000), count: 15, class_loss = 30.000002, iou_loss = nan, total_loss = nan 
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 150 Avg (IOU: 0.000000), count: 13, class_loss = 26.000000, iou_loss = 0.000025, total_loss = 26.000025 
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 161 Avg (IOU: 0.000000), count: 2, class_loss = 4.000000, iou_loss = 0.000031, total_loss = 4.000031 
 total_bbox = 831053, rewritten_bbox = 0.007460 % 
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 139 Avg (IOU: 0.000000), count: 15, class_loss = 30.000002, iou_loss = nan, total_loss = nan 
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 150 Avg (IOU: 0.000000), count: 6, clas

In [ ]:
# verilen inputun testi
!./darknet detector test "/content/darknet/obj.data" "/content/darknet/yolov4.cfg" "/content/gdrive/MyDrive/project/backup/yolov4_best.weights" "/content/gdrive/MyDrive/project/test-foto/test1.jpg" -dont_show